## Object Detection with YOLOv5

### About
----------------
This notebook provides a guided tour of deploying a YOLOv5 machine learning model pre-trained with MS COCO dataset using PyTorch to a Panorama appliance. More information about the model including the original model itself can be found in [this repository](https://github.com/ultralytics/yolov5) which is also included as a submodule under `3rdparty/yolov5`.

This example shows how to prepare a pre-trained model for deployment to a Panorama device. You can also train your own model using the resources from the aforementioned repository and deploy it to a Panorama appliance following the same steps.

This is an example of inference done on an image captured from a test IP camera.

![alt Test image inference results](test-result.png "Test image inference results")

### Imports & config
----------------

**We recommend running this notebook on SageMaker Notebook Instance or SageMaker Studio with `conda_python3` kernel as they come with many libraries used here pre-installed**. 

If you are not using Amazon SageMaker Notebook Instance or Studio then you need to install some additional libraries, like AWS Python SDK (`boto3`), etc.

In [ ]:
import sys
import time
import os
import shutil
import random as rnd
import json

from matplotlib import pyplot as plt
from IPython.display import Image
import numpy as np
import cv2
import torch
import boto3

import utils
print(f'Using torch {torch.__version__}')
print(f'Using python {sys.version_info}')

***Create your own AWS S3 Bucket making sure it contains `aws-panorama` in the bucket name***

In [ ]:
# Set this variable/constant value to be the full name of your bucket, for example "aws-panorama-example-xyz"
BUCKET = 'aws-panorama-<you-bucket-name-suffix>'  # Bucket name must contain "aws-panorama"

MODELS_S3_PREFIX = 'models'

MODEL = 'yolov5s'
model_file = f'{MODEL}.pt'
traced_model_file = f'{MODEL}.pth'
traced_model_archive = f'{MODEL}.tar.gz'

LAMBDA = 'yolov5s'

LAMBDA_EXECUTION_ROLE_NAME = 'PanoramaYoloLambdaExecutionRole'
lambda_file = f'{LAMBDA}_lambda.py'
lambda_archive = lambda_file.replace('.py', '.zip')

TEST_IMAGE = 'test.png'
INPUT_SIZE = 640 
THRESHOLD = 0.5
CLASSES = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 
           'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 
           'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 
           'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 
           'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 
           'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 
           'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 
           'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 
           'scissors', 'teddy bear', 'hair drier', 'toothbrush']
    
if torch.cuda.is_available():
    device_type = 'GPU'
    print(f'Found GPU: {torch.cuda.get_device_properties(0)}')
else:
    device_type = 'CPU'

# Uncomment next like if you want to force running on a CPU on a device with GPU
#device_type = 'CPU'

device = torch.device('cuda:0' if device_type == 'GPU' else 'cpu')
print(f'Using {device_type}', end='')

In [ ]:
s3_client = boto3.client("s3")
iam_client = boto3.client("iam")
lambda_client = boto3.client("lambda")

### The model
----------------

Model preparation steps are completed using the `3rdparty/yolov5` submodule

##### Steps to prepare the model
1. Download and trace the model
    - Install YOLOv5 dependencies
    - Run a test inference. This will also download the pre-trained model as save it as `yolov5s.pt` file
    - Export the downloaded model to TorchScript format, saved as `yolov5.pth` file
2. Test the TorchScript model
    - Load the TorchScript model
    - Load the test image and prepare it
    - Put the test image through the model
    - Show results
3. Pack and upload the TorchScript model to S3

##### Download and trace

Install extra dependencies required to execute YOLOv5 scripts

In [ ]:
!pip install -r 3rdparty/yolov5/requirements.txt

In [ ]:
device_str = 'cpu' if device_type == 'CPU' else '0'
out_dir = 'inference_results'
yolov4_dir = '3rdparty/yolov5'

if os.path.exists(out_dir):
    shutil.rmtree(out_dir)
    
!python $yolov4_dir/detect.py --weights $model_file --img $INPUT_SIZE --conf $THRESHOLD --source $TEST_IMAGE \
    --device $device_str --project $out_dir --name ""

!export PYTHONPATH=$yolov4_dir && python $yolov4_dir/models/export.py --weights $model_file --img-size $INPUT_SIZE
!mv yolov5s.torchscript.pt $traced_model_file

Image(filename=f'{out_dir}/{TEST_IMAGE}', width=600)

##### Test the traced model

Load the traced/exported model and a test image

In [ ]:
traced_model = torch.jit.load(traced_model_file, map_location=device)
test_image = cv2.cvtColor(cv2.imread(TEST_IMAGE), cv2.COLOR_BGR2RGB)

Note: output of the YOLOv5 model requires additional processing (Non Max Suppression) which can be done on GPU using PyTorch but on a Panorama appliance it needs to be executed on a CPU (also applies to execution of model's Detector layer logic), adding significant overhead to the overall inference process.

In [ ]:
processor = utils.Processor(CLASSES, INPUT_SIZE, threshold=THRESHOLD, keep_ratio=True)

tm = time.time()
img = torch.from_numpy(processor.preprocess(test_image)).to(device)
print(f'Pre-process: {int((time.time() - tm) * 1000)} msec')

# Do warm-up runs before timing inference
for i in range(3):
    traced_model(img)

run_count = 10
tm = time.time()
for i in range(run_count):
    results = traced_model(img)
print(f'Average inference time: {int((time.time() - tm) / run_count * 1000)} msec')
test_results = [r.cpu().numpy() for r in results]

tm = time.time()
_, result_image = processor.post_process(test_results, test_image.shape, test_image.copy())
print(f'Post-process: {int((time.time() - tm) * 1000)} msec')

# Show both the original and marked images
_, ax = plt.subplots(2, figsize=(10, 10))
ax[0].imshow(test_image)
ax[1].imshow(result_image)
plt.show()

##### Pack and upload the model archive to S3 bucket

Take a note of an S3 location of the uploaded model archive, you'll need it during the application creation process.

In [ ]:
!tar -czvf $traced_model_archive $traced_model_file

traced_model_key = os.path.join(MODELS_S3_PREFIX, traced_model_archive)
s3_client.upload_file(traced_model_archive, Bucket=BUCKET, Key=traced_model_key)

traced_model_s3_url = os.path.join(f's3://{BUCKET}', traced_model_key)
print(f'Uploaded model archive to {traced_model_s3_url}')

Alternatively, you can upload the model archive to S3 bucket using AWS Console or running the following AWS CLI command ***if you have AWS CLI installed and configured*** (change the cell to `Code` type before running)

### The Application 
---------------------

This is the script that will be deployed and executed on the Panorama Appliance as a lambda function. It can found in the same folder as this notebook along with another file `utils.py`, containing some helper scripts.

In [ ]:
!pygmentize $lambda_file

#### Create and deploy lambda

- If the execution of the code in this cell fails then make sure you have the rights to creates roles in AWS IAM
- **You only need to execute the next cell once.** All the subsequent deployments will use the same role 

In [ ]:
lambda_execution_role_policy = {
    "Version": "2012-10-17",
    "Statement":[
        {
            "Effect": "Allow",
            "Principal": {"Service": ["lambda.amazonaws.com", "events.amazonaws.com"]},
            "Action": "sts:AssumeRole",
        }
    ]
}
iam_client.create_role(
    RoleName=LAMBDA_EXECUTION_ROLE_NAME,
    AssumeRolePolicyDocument=json.dumps(lambda_execution_role_policy),
)

##### Create a new Lambda Function

*Use the cell in the [OPTIONAL] cell below if you want to re-deploy lambda after the initial deployment*

You can inspect the created AWS Lambda Function following the link shown after running the next cell

In [ ]:
!zip -o $lambda_archive $lambda_file utils.py

In [ ]:
# Uncomment and run the following if you already have a function and want to re-create it
# lambda_client.delete_function(FunctionName=LAMBDA)

with open(lambda_archive, "rb") as f:
    zipped_code = f.read()
    
lambda_execution_role = iam_client.get_role(RoleName=LAMBDA_EXECUTION_ROLE_NAME)

lambda_response = lambda_client.create_function(
    FunctionName=LAMBDA,
    Runtime="python3.7",
    Role=lambda_execution_role["Role"]["Arn"],
    Handler=lambda_file.replace('.py', '.main()'),
    Code=dict(ZipFile=zipped_code),
    Timeout=120,  
    MemorySize=2048,
    Publish=True)

template = "https://console.aws.amazon.com/lambda/home?region=us-east-1#/functions/{}/versions/{}?tab=configuration"
lambda_url = template.format(lambda_response["FunctionName"], lambda_response["Version"])
print(lambda_url)

##### [OPTIONAL] Subsequent deployments
Convert the next cell to Code type and run the following cell if you want to re-deploy the lambda function again, e.g. if you make changes to application code and want to deploy those changes to the Panorama appliance

### Deploy the Application to Panorama appliance 
---------------------

At the time of writing this the only way to deploy the Application to the Panorama device is through the [AWS Panorama Console](https://console.aws.amazon.com/panorama). Instructions for script-based deployment will be added here after alternative means of deployment are available, e.g. via AWS CLI or Python SDK.

Few things to remember when deploying the Application to your Panorama appliance in AWS Panorama Console:

- when creating a new model (as part of a new Application creation process) in AWS Panorama Console:
    - use the model archive uploaded to S3 earlier in this notebook to create a new **External model** (e.g. `s3://< your bucket >/models/yolov5s.tar.gz`)
    - make sure that the **Model name** you specify matches exactly the model name used in your Application/lambda code (it is stored in the variable/constant named **MODEL** in the current version of the Application/labmda code) *
    - select `PyTorch` as *Model framework*
    - add input with **Input name** `data` and **Input shape** `1,3,640,640`
     
- first deployment of the Application takes a bit longer due to initial conversion of the model done by AWS SageMaker Neo behind the scene. Subsequent deployments using the same model will be faster if you only change the Application code (which is usually the case)
- to troubleshoot any issues start with looking at the logs in [AWS CloudWatch](https://console.aws.amazon.com/cloudwatch). In the AWS CloudWatch Console, click on **Log Groups** under **Logs** and select a click on a link that has a name of the lambda function corresponding to your Application (something like `/aws/greengrass/Lambda/us-east-1/<your account number>/yolov5s`)

***Note:*** *code versions may change making it out of sync with comments in this notebook, always use the latest values from the code when referred to*

### What's next?
---------------------

This was just a taster to show you how to run a PyTorch based YOLOv5 model on Panorama appliance. Next logical step would be fine-tuning the pre-trained YOLOv5 model using your own dataset to recognise your own object types. Examples of doing that are available in the `3rdparty/yolov5` submodule.